# NTRU Encrypt

Num sistema criptográfico NTRU, **f** (e **g**, se necessário) são as chaves privadas, enquanto h
é a chave pública. Essas chaves podem ser geradas através do seguinte algoritmo:

## Algoritmo de Geração de chaves

INPUT: Um conjunto de parâmetros $Param = \{N, p, q, d\}$ e uma $seed$.

1. Instanciar $Sampler$ com $T(d + 1, d)$ e com a $seed$;
2. $f ← Sampler$
3. se $f$ não é invertível $mod q$ então retornar ao passo 2
4. $g ← Sampler$
5. $h = g/(pf + 1) mod q$

OUTPUT: Chave pública $h$ e chave secreta $(pf, g)$

## Algoritmo de Cifragem

INPUT: Chave pública h, mensagem msg de comprimento mlen, um conjunto de parâmetros $Param$ e a $seed$.
1. $m = Pad(msg, seed)$
2. $rseed = Hash(m|h)$
3. Instanciar $Sampler$ com $T$ e com $rseed$;
4. $r ← Sampler$
5. $t = r ∗ h$
6. $tseed = Hash(t)$
7. Instanciar $Sampler$ com $T$ e com $tseed$;
8. $m_{mask} ← Sampler$
9. $m' = m − m_{mask}(mod p)$
10. $c = t + m$

OUTPUT: Criptograma **c**

O algoritmo acima usa um **método de *padding*** para lidar com
entropia insuficiente potencial de uma mensagem. Supondo que o tamanho da mensagem é válido e menor que $(N - 173)$ *bits*, o algoritmo *padding* funciona da seguinte maneira:

1. Converter $msg$ numa *string* de *bits*. Cada *bit* forma um coeficiente binário para a parte inferior do polinómio $m$, partindo do coeficiente 0.

2. Os últimos $167$ coeficientes de $m (x)$ são escolhidos aleatoriamente de $\{−1, 0, 1\}$ (com um *input* *seed* ). O que dá mais de $256$ bits de entropia.

3. O comprimento da msg é convertido numa *string* binária de 8 *bits* e forma os últimos $173$ a $168$ coeficientes de $m (x)$.

## Algorithm de Decifragem

INPUT: Chave secreta $f$, chave pública $h$, criptograma $c$, e o conjunto de parâmetros $Param$.
1. $m' = f ∗ c (mod p)$
2. $t = c − m$
3. $tseed = Hash(t)$
4. Instanciar $Sampler$ com $T$ e com $tseed$;
5. $m_{mask} = Sampler$
6. $m = m' + m_{mask} (mod p)$
7. $rseed = Hash(m|h)$
8. Instanciar $Sampler$ com $T$ e com $rseed$;
9. $r ← Sampler$
10. $msg, mlen = Extract(m)$
11. se $p · r ∗ h = t$ então
12. $result = msg, mlen$
13. caso contrário
14. $result = ⊥ $

OUTPUT: $result$

No algoritmo acima a operação $Extrair ()$ corresponde ao inverso de $Pad ()$. Emite uma mensagem $m$ e seu comprimento $mlen$.

In [12]:
d = 10

N = next_prime(1 << d)
p = 3
q = next_prime(p*N)

print d, q, p, N

10 3109 3 1031


In [13]:
Z.<x>  = ZZ[]        # polinómios de coeficientes inteiros
Q.<x>  = PolynomialRing(GF(q),name='x').quotient(x^N-1); Q

Univariate Quotient Polynomial Ring in x over Finite Field of size 3109 with modulus x^1031 + 3108

In [14]:
def vec():
    return  [choice([-1,0,1]) for k in range(N)]

# arredondamento módulo 'q'

def qrnd(f):    # argumento em 'Q'
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

# arredondamento módulo 'p'

def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

def msg2bits(msg):
    '''
        Function that converts a message into a string of bits
    '''
    return bin(reduce(lambda x, y : (x<<8)+y, (ord(c) for c in msg), 1))[3:]

def padding(msg,seed):
    '''
        Padding method
    '''
    mlen = length(msg)
    if mlen < (N-173):
        msg = msg2bits(msg)
        m = []
        for i in range (0,mlen):
            
            #TODO
            choice([-1,0,1]) 
            


def extract(msg):
    '''
        Inverse of the padding function
    '''
    #TODO

In [23]:
import hashlib

class NTRU(object):
    def __init__(self,N,p,q,d,seed):
        # calcular um 'f' invertível
        f = Q(0)
        while not f.is_unit():
            F = Q(vec()); f = 1 + p*F
        G = Q(vec()); g = p*G
        # gerar as chaves      #10
        h = g^(-1) * (p*f + 1)
        self.chvpriv = (p*f,g)
        self.chvpub = h

    def encrypt(self,msg,param,seed):
        m = padding(msh,seed)

        sha256 = hashlib.sha256()
        sha256.update(m + self.chvpub)
        rseed = int(sha256.hexdigest(), 16)

        r = Q(vec())
        t = r*self.chvpub
        sha256.update(h)
        tseed = int(sha256.hexdigest(), 16)

        m_mask = Q(vec())
        m_2 = m - m_mask

        crip = t + m_2
        return crip


    def decrypt(self,crip,param):
        m_2 = Q(f * crip)
        t = crip - m
        sha256 = hashlib.sha256()
        sha256.update(h)
        tseed = int(sha256.hexdigest(), 16)

        m_mask = Q(vec())
        m = Q(m_2 + m_mask)

        sha256.update(m + self.chvpub)
        rseed = int(sha256.hexdigest(), 16)

        r = Q(vec())
        msg,mlen = m

        if p*r*self.chpub == t:
            result = msg,mlen
        else:
            result = "Error"
        return result

In [24]:
# Uma instância NTRU
K = NTRU(N,p,q,d,seed = vec())

# Uma mensagem aleatória
m = vec()
# Cifrar
e = K.encrypt(m,p,seed)  
# Decifrar e Verificar
m == K.decrypt(e,p)

TypeError: can only concatenate list (not "type") to list